In [9]:
import sys
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.signals import signals

In [11]:
sys.path

['c:\\Users\\stew1\\Documents\\GitHub_Repos\\FinTech_Projects\\Crypto_Bandyts\\libs\\notebooks',
 'c:\\Users\\stew1\\.vscode\\extensions\\ms-toolsai.jupyter-2020.12.414227025\\pythonFiles\\vscode_datascience_helpers\\..\\.does-not-exist',
 'c:\\Users\\stew1\\.vscode\\extensions\\ms-toolsai.jupyter-2020.12.414227025\\pythonFiles',
 'c:\\Users\\stew1\\.vscode\\extensions\\ms-toolsai.jupyter-2020.12.414227025\\pythonFiles\\lib\\python',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\python37.zip',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\DLLs',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\lib',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev',
 '',
 'C:\\Users\\stew1\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\lib\\site-packages',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\lib\\site-packages\\win32',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\stew1\\Anaconda3\\envs\\dev\\lib\\site-packages\\Pythonwin',
 'C:\\